In [2]:
import pandas as pd
import numpy as np
import openpyxl
import datetime
import cx_Oracle as cx
#from sqlalchemy import create_engine

In [246]:
dictionary = pd.read_csv('字典2.csv')
employee = pd.read_excel('员工学历.xlsx')

In [247]:
employee = employee[["ITEM_NAME", "STAFF_NUMBER", "ANN_DT", "S_INFO_WINDCODE"]]
dictionary = dictionary[["ITEM_NAME", "分数"]]
dictionary.head()

,ITEM_NAME,分数
0,(中专及以下),10.5
1,EMBA,18.5
2,EMBA在读,18.5
3,MBA,18.5
4,一线,12.0


In [248]:
employee.head()

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE
0,博士,28,20100324,000686.SZ
1,大专以上,23698,20100420,600016.SH
2,本科,4745,20100428,600837.SH
3,硕士,133,20100424,601099.SH
4,大专,4715,20100304,601166.SH


In [249]:
employee_1 = employee.groupby(["S_INFO_WINDCODE","ANN_DT"])


In [250]:
employee_1.tail(20)

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE
0,博士,28,20100324,000686.SZ
1,大专以上,23698,20100420,600016.SH
2,本科,4745,20100428,600837.SH
3,硕士,133,20100424,601099.SH
4,大专,4715,20100304,601166.SH
...,...,...,...,...
194869,专科,51,20170422,688555.SH
194870,专科以下,7,20170422,688555.SH
194871,博士,2,20170422,688555.SH
194872,本科,91,20170422,688555.SH


In [251]:
employee_grade = pd.merge(employee,dictionary,how = "inner", left_on = "ITEM_NAME", right_on = "ITEM_NAME")
employee_grade.head(5)

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数
0,博士,28,20100324,000686.SZ,22.0
1,博士,1,20140403,600461.SH,22.0
2,博士,2,20140416,600661.SH,22.0
3,博士,1,20120910,603993.SH,22.0
4,博士,1,20150331,300327.SZ,22.0


In [252]:
employee_grade['ANN_DT'] = pd.to_datetime(employee_grade['ANN_DT'], format='%Y%m%d')
employee_grade

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数
0,博士,28,2010-03-24,000686.SZ,22.0
1,博士,1,2014-04-03,600461.SH,22.0
2,博士,2,2014-04-16,600661.SH,22.0
3,博士,1,2012-09-10,603993.SH,22.0
4,博士,1,2015-03-31,300327.SZ,22.0
...,...,...,...,...,...
194854,高中/职高及以下,411,2022-04-27,000667.SZ,10.0
194855,高中/职高及以下,334,2023-04-29,000667.SZ,10.0
194856,大专以下(含),1525,2023-04-27,300056.SZ,12.0
194857,本科以下人员,7000,2017-04-26,600466.SH,12.5


In [62]:
example = employee_grade.groupby(["S_INFO_WINDCODE","ANN_DT"])["ITEM_NAME"].count()
example

S_INFO_WINDCODE  ANN_DT    
000001.SZ        2010-03-12    2
                 2012-03-09    3
                 2012-08-16    3
                 2013-03-08    3
                 2013-08-23    3
                              ..
873576.BJ        2023-04-01    5
873593.BJ        2020-12-30    3
                 2022-04-28    4
                 2022-08-30    4
                 2023-03-18    4
Name: ITEM_NAME, Length: 48744, dtype: int64

In [71]:
df = example.reset_index(name='ITEM_NAME')

In [72]:
df = df[["S_INFO_WINDCODE", "ANN_DT"]]

In [77]:
df

,S_INFO_WINDCODE,ANN_DT
0,000001.SZ,2010-03-12
1,000001.SZ,2012-03-09
2,000001.SZ,2012-08-16
3,000001.SZ,2013-03-08
4,000001.SZ,2013-08-23
...,...,...
48739,873576.BJ,2023-04-01
48740,873593.BJ,2020-12-30
48741,873593.BJ,2022-04-28
48742,873593.BJ,2022-08-30


In [81]:
min_date = df['ANN_DT'].min()
max_date = df['ANN_DT'].max()
min_date

Timestamp('1998-12-31 00:00:00')

In [82]:
max_date

Timestamp('2023-07-31 00:00:00')

In [80]:
df['ANN_DT'] = pd.to_datetime(df['ANN_DT'])

In [83]:
df_1 = df[df['ANN_DT']<='1999-04-30']
df_1

,S_INFO_WINDCODE,ANN_DT
33664,600690.SH,1998-12-31


In [89]:
df_2 = df[(df['ANN_DT']>'1999-10-31') & (df['ANN_DT']<='2000-04-30')]
df_2

,S_INFO_WINDCODE,ANN_DT
2631,000651.SZ,1999-12-31
27733,600104.SH,1999-12-31
33665,600690.SH,1999-12-31


In [91]:
df_3 = df[(df['ANN_DT']>'2000-10-31') & (df['ANN_DT']<='2001-04-30')]
df_3

,S_INFO_WINDCODE,ANN_DT
2632,000651.SZ,2001-03-10
33666,600690.SH,2001-03-16


In [93]:
df_4 = df[(df['ANN_DT']>'2001-10-31') & (df['ANN_DT']<='2002-04-30')]
df_4

,S_INFO_WINDCODE,ANN_DT
2633,000651.SZ,2002-03-16
27734,600104.SH,2002-03-26
33667,600690.SH,2002-03-29


In [95]:
df_5 = df[(df['ANN_DT']>'2002-10-31') & (df['ANN_DT']<='2003-04-30')]
df_5

,S_INFO_WINDCODE,ANN_DT
2634,000651.SZ,2003-04-19
27735,600104.SH,2003-03-26
33668,600690.SH,2003-03-28


In [100]:
df_6 = df[(df['ANN_DT']>'2003-10-31') & (df['ANN_DT']<='2004-04-30')]
df_6

,S_INFO_WINDCODE,ANN_DT
2635,000651.SZ,2004-04-23
27736,600104.SH,2004-03-03
33669,600690.SH,2004-03-26


In [101]:
df_7 = df[(df['ANN_DT']>'2004-10-31') & (df['ANN_DT']<='2005-04-30')]
df_7

,S_INFO_WINDCODE,ANN_DT
2636,000651.SZ,2005-02-24
27737,600104.SH,2005-02-05
33670,600690.SH,2005-04-08


In [103]:
df_8 = df[(df['ANN_DT']>'2005-10-31') & (df['ANN_DT']<='2006-04-30')]
df_8

,S_INFO_WINDCODE,ANN_DT
2637,000651.SZ,2006-04-11
27738,600104.SH,2006-03-24
33671,600690.SH,2006-04-27


In [105]:
df_9 = df[(df['ANN_DT']>'2006-10-31') & (df['ANN_DT']<='2007-04-30')]
df_9

,S_INFO_WINDCODE,ANN_DT
2638,000651.SZ,2007-04-12
27739,600104.SH,2007-03-30
33672,600690.SH,2007-04-27


In [107]:
df_10 = df[(df['ANN_DT']>'2007-10-31') & (df['ANN_DT']<='2008-04-30')]
df_10

,S_INFO_WINDCODE,ANN_DT
743,000100.SZ,2008-03-14
2639,000651.SZ,2008-04-18
31578,600498.SH,2008-04-24


In [114]:
df_11 = df[(df['ANN_DT']>'2008-10-31') & (df['ANN_DT']<='2009-04-30')]
df_11

,S_INFO_WINDCODE,ANN_DT
744,000100.SZ,2009-03-26
2640,000651.SZ,2009-04-21
31579,600498.SH,2009-04-23


In [144]:
df_12 = df[(df['ANN_DT']>'2009-10-31') & (df['ANN_DT']<='2010-04-30')]
df_12.head()

,S_INFO_WINDCODE,ANN_DT
0,000001.SZ,2010-03-12
745,000100.SZ,2010-03-11
1868,000562.SZ,2010-03-19
1873,000563.SZ,2010-04-26
2641,000651.SZ,2010-04-27


In [151]:
# 使用groupby和agg函数找到每个代码的最大公告日期
max_dates_12 = df_12.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_12_df = max_dates_12.reset_index(name="MAX_ANN_DT")
max_dates_12_df.head()

,S_INFO_WINDCODE,MAX_ANN_DT
0,000001.SZ,2010-03-12
1,000100.SZ,2010-03-11
2,000562.SZ,2010-03-19
3,000563.SZ,2010-04-26
4,000651.SZ,2010-04-27


In [145]:
df_39 = df[(df['ANN_DT']>'2010-04-30') & (df['ANN_DT']<='2010-10-31')]
df_39.head()

,S_INFO_WINDCODE,ANN_DT
11902,002500.SZ,2010-09-17
37450,601288.SH,2010-08-03
37625,601377.SH,2010-09-14
38330,601818.SH,2010-08-17


In [152]:
# 使用groupby和agg函数找到每个代码的最大公告日期
max_dates_39 = df_39.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_39_df = max_dates_39.reset_index(name="MAX_ANN_DT")
max_dates_39_df.head()

,S_INFO_WINDCODE,MAX_ANN_DT
0,002500.SZ,2010-09-17
1,601288.SH,2010-08-03
2,601377.SH,2010-09-14
3,601818.SH,2010-08-17


In [116]:
df_13 = df[(df['ANN_DT']>'2010-10-31') & (df['ANN_DT']<='2011-04-30')]
df_13.head()

,S_INFO_WINDCODE,ANN_DT
1869,000562.SZ,2011-03-29
1874,000563.SZ,2011-01-18
2642,000651.SZ,2011-03-24
2974,000686.SZ,2011-04-01
3390,000728.SZ,2011-03-22


In [117]:
df_14 = df[(df['ANN_DT']>'2011-04-30') & (df['ANN_DT']<='2011-10-31')]
df_14.head()

,S_INFO_WINDCODE,ANN_DT
12893,002584.SZ,2011-05-17
12906,002585.SZ,2011-05-17
12919,002586.SZ,2011-05-17
12932,002587.SZ,2011-05-24
12945,002588.SZ,2011-05-24


In [118]:
df_15 = df[(df['ANN_DT']>'2011-10-31') & (df['ANN_DT']<='2012-04-30')]
df_15.head()

,S_INFO_WINDCODE,ANN_DT
1,000001.SZ,2012-03-09
20,000002.SZ,2012-03-13
44,000004.SZ,2012-04-21
56,000005.SZ,2012-04-24
68,000006.SZ,2012-04-18


In [119]:
df_16 = df[(df['ANN_DT']>'2012-04-30') & (df['ANN_DT']<='2012-10-31')]
df_16.head()

,S_INFO_WINDCODE,ANN_DT
2,000001.SZ,2012-08-16
1876,000563.SZ,2012-08-28
3392,000728.SZ,2012-08-28
3526,000750.SZ,2012-08-24
3722,000776.SZ,2012-08-28


In [120]:
df_17 = df[(df['ANN_DT']>'2012-10-31') & (df['ANN_DT']<='2013-04-30')]
df_17.head()

,S_INFO_WINDCODE,ANN_DT
3,000001.SZ,2013-03-08
21,000002.SZ,2013-02-28
45,000004.SZ,2013-04-20
57,000005.SZ,2013-04-24
69,000006.SZ,2013-03-30


In [153]:
# 使用groupby和agg函数找到每个代码的最大公告日期
max_dates_13 = df_13.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_13_df = max_dates_13.reset_index(name="MAX_ANN_DT")

max_dates_14 = df_14.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_14_df = max_dates_14.reset_index(name="MAX_ANN_DT")

max_dates_15 = df_15.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_15_df = max_dates_15.reset_index(name="MAX_ANN_DT")

max_dates_16 = df_16.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_16_df = max_dates_16.reset_index(name="MAX_ANN_DT")

max_dates_17 = df_17.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_17_df = max_dates_17.reset_index(name="MAX_ANN_DT")

In [121]:
df_18 = df[(df['ANN_DT']>'2013-04-30') & (df['ANN_DT']<='2013-10-31')]
df_18.head()

,S_INFO_WINDCODE,ANN_DT
4,000001.SZ,2013-08-23
26699,600000.SH,2013-08-14
26840,600016.SH,2013-08-29
27057,600036.SH,2013-08-17
29764,600298.SH,2013-05-31


In [122]:
df_19 = df[(df['ANN_DT']>'2013-10-31') & (df['ANN_DT']<='2014-04-30')]
df_19.head()

,S_INFO_WINDCODE,ANN_DT
5,000001.SZ,2014-03-07
22,000002.SZ,2014-03-07
46,000004.SZ,2014-04-22
58,000005.SZ,2014-04-26
70,000006.SZ,2014-03-15


In [123]:
df_20 = df[(df['ANN_DT']>'2014-04-30') & (df['ANN_DT']<='2014-10-31')]
df_20.head()

,S_INFO_WINDCODE,ANN_DT
6,000001.SZ,2014-08-14
23,000002.SZ,2014-08-18
3588,000756.SZ,2014-08-01
4712,000898.SZ,2014-08-14
13740,002649.SZ,2014-08-21


In [124]:
df_21 = df[(df['ANN_DT']>'2014-10-31') & (df['ANN_DT']<='2015-04-30')]
df_21.head()

,S_INFO_WINDCODE,ANN_DT
7,000001.SZ,2015-03-13
24,000002.SZ,2015-03-31
47,000004.SZ,2015-04-30
59,000005.SZ,2015-04-11
71,000006.SZ,2015-04-01


In [125]:
df_22 = df[(df['ANN_DT']>'2015-04-30') & (df['ANN_DT']<='2015-10-31')]
df_22.head()

,S_INFO_WINDCODE,ANN_DT
8,000001.SZ,2015-08-14
25,000002.SZ,2015-08-17
2858,000673.SZ,2015-05-05
3590,000756.SZ,2015-07-31
5403,000972.SZ,2015-05-04


In [126]:
df_23 = df[(df['ANN_DT']>'2015-10-31') & (df['ANN_DT']<='2016-04-30')]
df_23.head()

,S_INFO_WINDCODE,ANN_DT
9,000001.SZ,2016-03-10
26,000002.SZ,2016-03-14
48,000004.SZ,2016-04-30
60,000005.SZ,2016-04-27
72,000006.SZ,2016-03-30


In [127]:
df_24 = df[(df['ANN_DT']>'2016-04-30') & (df['ANN_DT']<='2016-10-31')]
df_24.head()

,S_INFO_WINDCODE,ANN_DT
10,000001.SZ,2016-08-12
27,000002.SZ,2016-08-22
3095,000699.SZ,2016-07-30
3592,000756.SZ,2016-08-30
4715,000898.SZ,2016-08-30


In [128]:
df_25 = df[(df['ANN_DT']>'2016-10-31') & (df['ANN_DT']<='2017-04-30')]
df_25.head()

,S_INFO_WINDCODE,ANN_DT
11,000001.SZ,2017-03-17
28,000002.SZ,2017-03-27
49,000004.SZ,2017-04-11
61,000005.SZ,2017-04-27
73,000006.SZ,2017-03-30


In [129]:
df_26 = df[(df['ANN_DT']>'2017-04-30') & (df['ANN_DT']<='2017-10-31')]
df_26.head()

,S_INFO_WINDCODE,ANN_DT
12,000001.SZ,2017-08-11
29,000002.SZ,2017-08-25
1338,000511.SZ,2017-06-24
2702,000658.SZ,2017-06-10
3594,000756.SZ,2017-08-14


In [130]:
df_27 = df[(df['ANN_DT']>'2017-10-31') & (df['ANN_DT']<='2018-04-30')]
df_27.head()

,S_INFO_WINDCODE,ANN_DT
13,000001.SZ,2018-03-15
30,000002.SZ,2018-03-27
38,000003.SZ,2018-04-27
50,000004.SZ,2018-04-28
62,000005.SZ,2018-04-21


In [131]:
df_28 = df[(df['ANN_DT']>'2018-04-30') & (df['ANN_DT']<='2018-10-31')]
df_28.head()

,S_INFO_WINDCODE,ANN_DT
14,000001.SZ,2018-08-16
31,000002.SZ,2018-08-21
479,000047.SZ,2018-07-19
487,000048.SZ,2018-08-31
961,000405.SZ,2018-08-21


In [132]:
df_29 = df[(df['ANN_DT']>'2018-10-31') & (df['ANN_DT']<='2019-04-30')]
df_29.head()

,S_INFO_WINDCODE,ANN_DT
15,000001.SZ,2019-03-07
32,000002.SZ,2019-03-26
39,000003.SZ,2019-04-27
51,000004.SZ,2019-04-26
63,000005.SZ,2019-04-25


In [133]:
df_30 = df[(df['ANN_DT']>'2019-04-30') & (df['ANN_DT']<='2019-10-31')]
df_30.head()

,S_INFO_WINDCODE,ANN_DT
16,000001.SZ,2019-08-08
33,000002.SZ,2019-08-21
963,000405.SZ,2019-08-28
1578,000535.SZ,2019-08-29
1802,000556.SZ,2019-08-28


In [134]:
df_31 = df[(df['ANN_DT']>'2019-10-31') & (df['ANN_DT']<='2020-04-30')]
df_31.head()

,S_INFO_WINDCODE,ANN_DT
17,000001.SZ,2020-02-14
34,000002.SZ,2020-03-17
52,000004.SZ,2020-04-28
76,000006.SZ,2020-04-09
88,000007.SZ,2020-04-29


In [136]:
df_32 = df[(df['ANN_DT']>'2020-04-30') & (df['ANN_DT']<='2020-10-31')]
df_32.head()

,S_INFO_WINDCODE,ANN_DT
18,000001.SZ,2020-08-28
35,000002.SZ,2020-08-28
40,000003.SZ,2020-05-01
64,000005.SZ,2020-05-23
198,000018.SZ,2020-05-30


In [137]:
df_33 = df[(df['ANN_DT']>'2020-10-31') & (df['ANN_DT']<='2021-04-30')]
df_33.head()

,S_INFO_WINDCODE,ANN_DT
19,000001.SZ,2021-02-02
36,000002.SZ,2021-03-31
53,000004.SZ,2021-04-28
65,000005.SZ,2021-04-30
77,000006.SZ,2021-03-31


In [138]:
df_34 = df[(df['ANN_DT']>'2021-04-30') & (df['ANN_DT']<='2021-10-31')]
df_34.head()

,S_INFO_WINDCODE,ANN_DT
41,000003.SZ,2021-05-01
199,000018.SZ,2021-05-01
2728,000660.SZ,2021-09-01
3100,000699.SZ,2021-07-01
3602,000756.SZ,2021-08-30


In [139]:
df_35 = df[(df['ANN_DT']>'2021-10-31') & (df['ANN_DT']<='2022-04-30')]
df_35.head()

,S_INFO_WINDCODE,ANN_DT
37,000002.SZ,2022-03-31
42,000003.SZ,2022-04-29
54,000004.SZ,2022-04-30
66,000005.SZ,2022-04-27
78,000006.SZ,2022-04-16


In [140]:
df_36 = df[(df['ANN_DT']>'2022-04-30') & (df['ANN_DT']<='2022-10-31')]
df_36.head()

,S_INFO_WINDCODE,ANN_DT
200,000018.SZ,2022-09-01
1106,000420.SZ,2022-05-28
3101,000699.SZ,2022-06-30
3604,000756.SZ,2022-08-31
3837,000787.SZ,2022-06-01


In [141]:
df_37 = df[(df['ANN_DT']>'2022-10-31') & (df['ANN_DT']<='2023-04-30')]
df_37.head()

,S_INFO_WINDCODE,ANN_DT
43,000003.SZ,2023-04-29
55,000004.SZ,2023-04-29
67,000005.SZ,2023-04-29
79,000006.SZ,2023-04-28
91,000007.SZ,2023-04-29


In [142]:
df_38 = df[(df['ANN_DT']>'2023-04-30') & (df['ANN_DT']<='2023-10-31')]
df_38.head()

,S_INFO_WINDCODE,ANN_DT
2084,000587.SZ,2023-06-13
2708,000658.SZ,2023-06-01
2866,000673.SZ,2023-06-14
5896,001373.SZ,2023-05-18
9047,002260.SZ,2023-06-16


In [160]:
# 使用groupby和agg函数找到每个代码的最大公告日期
max_dates_18 = df_18.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_18_df = max_dates_18.reset_index(name="MAX_ANN_DT")

max_dates_19 = df_19.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_19_df = max_dates_19.reset_index(name="MAX_ANN_DT")

max_dates_20 = df_20.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_20_df = max_dates_20.reset_index(name="MAX_ANN_DT")

max_dates_21 = df_21.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_21_df = max_dates_21.reset_index(name="MAX_ANN_DT")

max_dates_22 = df_22.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_22_df = max_dates_22.reset_index(name="MAX_ANN_DT")

max_dates_23 = df_23.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_23_df = max_dates_23.reset_index(name="MAX_ANN_DT")

max_dates_24 = df_24.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_24_df = max_dates_24.reset_index(name="MAX_ANN_DT")

max_dates_25 = df_25.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_25_df = max_dates_25.reset_index(name="MAX_ANN_DT")

max_dates_26 = df_26.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_26_df = max_dates_26.reset_index(name="MAX_ANN_DT")

max_dates_27 = df_27.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_27_df = max_dates_27.reset_index(name="MAX_ANN_DT")

max_dates_28 = df_28.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_28_df = max_dates_28.reset_index(name="MAX_ANN_DT")

max_dates_29 = df_29.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_29_df = max_dates_29.reset_index(name="MAX_ANN_DT")

max_dates_30 = df_30.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_30_df = max_dates_30.reset_index(name="MAX_ANN_DT")

max_dates_31 = df_31.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_31_df = max_dates_31.reset_index(name="MAX_ANN_DT")

max_dates_32 = df_32.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_32_df = max_dates_32.reset_index(name="MAX_ANN_DT")

max_dates_33 = df_33.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_33_df = max_dates_33.reset_index(name="MAX_ANN_DT")

max_dates_34 = df_34.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_34_df = max_dates_34.reset_index(name="MAX_ANN_DT")

max_dates_35 = df_35.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_35_df = max_dates_35.reset_index(name="MAX_ANN_DT")

max_dates_36 = df_36.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_36_df = max_dates_36.reset_index(name="MAX_ANN_DT")

max_dates_37 = df_37.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_37_df = max_dates_37.reset_index(name="MAX_ANN_DT")

max_dates_38 = df_38.groupby("S_INFO_WINDCODE")["ANN_DT"].max()
max_dates_38_df = max_dates_38.reset_index(name="MAX_ANN_DT")

In [163]:
df_combine_0 = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11], axis = 0).reset_index(drop=True)
df_combine_0 = df_combine_0.rename(columns={"ANN_DT": "MAX_ANN_DT"}, inplace=True)
df_combine = pd.concat([df_combine_0,max_dates_12_df,max_dates_39_df,max_dates_13_df,max_dates_14_df,max_dates_15_df,max_dates_16_df,max_dates_17_df,max_dates_18_df,max_dates_19_df,max_dates_20_df,max_dates_21_df,max_dates_22_df,max_dates_23_df,max_dates_24_df,max_dates_25_df,max_dates_26_df,max_dates_27_df,max_dates_28_df,max_dates_29_df,max_dates_30_df,max_dates_31_df,max_dates_32_df,max_dates_33_df,max_dates_34_df,max_dates_35_df,max_dates_36_df,max_dates_37_df,max_dates_38_df], axis = 0).reset_index(drop=True)
df_combine

,S_INFO_WINDCODE,MAX_ANN_DT
0,000001.SZ,2010-03-12
1,000100.SZ,2010-03-11
2,000562.SZ,2010-03-19
3,000563.SZ,2010-04-26
4,000651.SZ,2010-04-27
...,...,...
47748,688646.SH,2023-05-08
47749,688651.SH,2023-05-17
47750,688693.SH,2023-07-14
47751,830839.BJ,2023-07-21


In [253]:
#合并
final_df = pd.merge(employee_grade,df_combine,how = "inner", left_on = ["S_INFO_WINDCODE","ANN_DT"], right_on = ["S_INFO_WINDCODE","MAX_ANN_DT"])
final_df

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,博士,28,2010-03-24,000686.SZ,22.00,2010-03-24
1,本科,1608,2010-03-24,000686.SZ,16.00,2010-03-24
2,硕士,279,2010-03-24,000686.SZ,18.50,2010-03-24
3,大专,836,2010-03-24,000686.SZ,15.00,2010-03-24
4,中专及以下,144,2010-03-24,000686.SZ,10.00,2010-03-24
...,...,...,...,...,...,...
190640,具有中专及高中学历,115,2023-04-25,300837.SZ,12.00,2023-04-25
190641,具有初中及以以下学历,212,2023-04-25,300837.SZ,7.00,2023-04-25
190642,具有大学专科及以下学历人员,5416,2023-04-27,002758.SZ,12.00,2023-04-27
190643,具有研究生及以上学历(位)人员,104,2023-04-27,002758.SZ,20.25,2023-04-27


In [254]:
final_df.to_csv('final_df.csv', encoding='utf-8_sig', index=True)

In [222]:
#oooo = final_df.loc[final_df["分数"] == "NAN"]
#oooo

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数,MAX_ANN_DT
27,其他学历,755,2010-04-08,601628.SH,NAN,2010-04-08
38,其他学历,7729,2010-04-17,601318.SH,NAN,2010-04-17
43,其他人员,1227,2013-03-28,000400.SZ,NAN,2013-03-28
72,其他人员,1104,2012-03-09,600495.SH,NAN,2012-03-09
108,其他人员,153,2013-04-18,300098.SZ,NAN,2013-04-18
...,...,...,...,...,...,...
190188,其他,1631,2023-04-29,002440.SZ,NAN,2023-04-29
190228,其他,569,2023-04-28,300429.SZ,NAN,2023-04-28
190410,其他,9,2023-04-27,300588.SZ,NAN,2023-04-27
190470,其他,2430,2023-04-28,002418.SZ,NAN,2023-04-28


In [271]:
final_df_new = final_df.sort_values(by = ["S_INFO_WINDCODE","MAX_ANN_DT"],axis=0, ascending=True)
final_df_new = final_df_new.reset_index()
final_df_new

,index,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,38389,大专以上,10765,2010-03-12,000001.SZ,18.5,2010-03-12
1,38390,本科以上,8040,2010-03-12,000001.SZ,19.0,2010-03-12
2,121995,大专,2866,2012-03-09,000001.SZ,15.0,2012-03-09
3,121996,其他,454,2012-03-09,000001.SZ,NaN,2012-03-09
4,121997,本科及以上,10458,2012-03-09,000001.SZ,19.0,2012-03-09
...,...,...,...,...,...,...,...
190640,106787,专科,56,2022-08-30,873593.BJ,15.0,2022-08-30
190641,106969,本科,75,2023-03-18,873593.BJ,16.0,2023-03-18
190642,106970,硕士,5,2023-03-18,873593.BJ,18.5,2023-03-18
190643,106971,专科以下,78,2023-03-18,873593.BJ,12.0,2023-03-18


In [ ]:
'''for i in len(final_df):
    windcode_data = pd.DataFrame()
    if final_df_new['S_INFO_WINDCODE'].loc[i] == final_df_new['S_INFO_WINDCODE'].loc[i+1] and final_df_new['MAX_ANN_DT'].loc[i] == final_df_new['MAX_ANN_DT'].loc[i+1]:
        windcode_data = windcode_data.append(final_df_new.loc[i])
    windcode_data = windcode_data.append(final_df_new.loc[i+1])
    for j in len(windcode_data):'''
        

In [ ]:
for i in len(final_df):
    if final_df_new["ITEM_NAME"].loc[i] = "其他":
        same_code_dt_rows = final_df_new[(final_df_new['S_INFO_WINDCODE'] == row['S_INFO_WINDCODE']) & (final_df_new['MAX_ANN_DT'] == row['MAX_ANN_DT'])]

In [263]:
# 找到分数中为"NAN"的行
nan_rows = final_df_new[final_df_new['分数'].isna()]

# 遍历每一行，根据条件替换分数中的"NAN"
for index, row in nan_rows.iterrows():
    # 找到相同S_INFO_WINDCODE和MAX_ANN_DT的其他行
    same_code_dt_rows = final_df_new[(final_df_new['S_INFO_WINDCODE'] == row['S_INFO_WINDCODE']) & (final_df_new['MAX_ANN_DT'] == row['MAX_ANN_DT'])]
    
    # 根据条件进行分数的替换
    if any(item_name in row['ITEM_NAME'] for item_name in ['高中', '中专', '中技', '中等']):
        final_df_new.at[index, '分数'] = 10
    elif any(item_name in row['ITEM_NAME'] for item_name in ['大专', '专科']):
        final_df_new.at[index, '分数'] = 12

In [288]:
nan_rows = final_df_new[final_df_new['分数'].isna()]
nan_rows

,index,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数,MAX_ANN_DT
3,121996,其他,454,2012-03-09,000001.SZ,NaN,2012-03-09
6,121139,其他,664,2012-08-16,000001.SZ,NaN,2012-08-16
9,120463,其他,727,2013-03-08,000001.SZ,NaN,2013-03-08
12,130622,其他,690,2013-08-23,000001.SZ,NaN,2013-08-23
15,121152,其他,701,2014-03-07,000001.SZ,NaN,2014-03-07
...,...,...,...,...,...,...,...
188804,172303,其他,22,2016-07-26,839719.BJ,NaN,2016-07-26
189145,90312,其他,202,2016-08-30,870204.BJ,NaN,2016-08-30
189921,100270,其他,65,2017-04-28,871970.BJ,NaN,2017-04-28
190059,104840,其他,19,2017-05-26,872351.BJ,NaN,2017-05-26


In [314]:
a = '高中'
b = '中专'
c = '中技'
d = '中等'
e = '大专'
f = '专科'

for index, row in nan_rows.iterrows():
    # 找到相同S_INFO_WINDCODE和MAX_ANN_DT的其他行
    same_code_dt_rows = final_df_new[(final_df_new['S_INFO_WINDCODE'] == row['S_INFO_WINDCODE']) & (final_df_new['MAX_ANN_DT'] == row['MAX_ANN_DT'])]
    same_code_dt_rows = same_code_dt_rows.reset_index(drop = True)
    nan_row = same_code_dt_rows[same_code_dt_rows['分数'].isna()]
    nan_row = nan_row.reset_index(drop = True)
    
    for i in range(len(same_code_dt_rows)):
        if (a in same_code_dt_rows['ITEM_NAME'].loc[i]) or (b in same_code_dt_rows['ITEM_NAME'].loc[i]) or (c in same_code_dt_rows['ITEM_NAME'].loc[i]) or (d in same_code_dt_rows['ITEM_NAME'].loc[i]):
            for j in range(len(final_df_new)):
                if nan_row['S_INFO_WINDCODE'].loc[0] == final_df_new['S_INFO_WINDCODE'].loc[j] and nan_row['MAX_ANN_DT'].loc[0] == final_df_new['MAX_ANN_DT'].loc[j] and nan_row['ITEM_NAME'].loc[0] == final_df_new['ITEM_NAME'].loc[j]:
                    final_df_new['分数'].loc[j] = 10
        elif (e in same_code_dt_rows['ITEM_NAME'].loc[i]) or (f in same_code_dt_rows['ITEM_NAME'].loc[i]):
            for j in range(len(final_df_new)):
                if nan_row['S_INFO_WINDCODE'].loc[0] == final_df_new['S_INFO_WINDCODE'].loc[j] and nan_row['MAX_ANN_DT'].loc[0] == final_df_new['MAX_ANN_DT'].loc[j] and nan_row['ITEM_NAME'].loc[0] == final_df_new['ITEM_NAME'].loc[j]:
                    final_df_new['分数'].loc[j] = 12
        

/var/folders/7b/33w4_t297zxfr68lt1r2hm9c0000gn/T/ipykernel_19286/4274637839.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_new['分数'].loc[j] = 12
/var/folders/7b/33w4_t297zxfr68lt1r2hm9c0000gn/T/ipykernel_19286/4274637839.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_new['分数'].loc[j] = 10


KeyError: 0

In [25]:
final_df_new

,Unnamed: 0,index,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,0,38389,大专以上,10765,2010-03-12,000001.SZ,18.5,2010-03-12
1,1,38390,本科以上,8040,2010-03-12,000001.SZ,19.0,2010-03-12
2,2,121995,大专,2866,2012-03-09,000001.SZ,15.0,2012-03-09
3,3,121996,其他,454,2012-03-09,000001.SZ,12.0,2012-03-09
4,4,121997,本科及以上,10458,2012-03-09,000001.SZ,19.0,2012-03-09
...,...,...,...,...,...,...,...,...
190640,190640,106787,专科,56,2022-08-30,873593.BJ,15.0,2022-08-30
190641,190641,106969,本科,75,2023-03-18,873593.BJ,16.0,2023-03-18
190642,190642,106970,硕士,5,2023-03-18,873593.BJ,18.5,2023-03-18
190643,190643,106971,专科以下,78,2023-03-18,873593.BJ,12.0,2023-03-18


In [317]:
final_df_new.to_csv('final_df_new.csv', encoding='utf-8_sig', index=True)

In [31]:
final_df_new = pd.read_csv('final_df_new.csv', encoding='utf-8_sig')

In [32]:
nan_rows_new = final_df_new[final_df_new['分数'].isna()].reset_index(drop = True)
nan_rows_new = nan_rows_new[['ITEM_NAME', 'STAFF_NUMBER', 'S_INFO_WINDCODE', '分数', 'MAX_ANN_DT']]
nan_rows_new

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,其他人员,196,000005.SZ,NaN,2012-04-24
1,其他人员,81,000006.SZ,NaN,2013-03-30
2,其他人员,14,000008.SZ,NaN,2012-03-03
3,其他人员,20,000010.SZ,NaN,2012-04-27
4,其他人员,1206,000011.SZ,NaN,2012-03-30
...,...,...,...,...,...
4165,其他,22,839719.BJ,NaN,2016-07-26
4166,其他,202,870204.BJ,NaN,2016-08-30
4167,其他,65,871970.BJ,NaN,2017-04-28
4168,其他,19,872351.BJ,NaN,2017-05-26


In [34]:
for index, row in nan_rows_new.iterrows():
    # 找到相同S_INFO_WINDCODE和MAX_ANN_DT的其他行
    same_code_dt_rows = final_df_new[(final_df_new['S_INFO_WINDCODE'] == row['S_INFO_WINDCODE']) & (final_df_new['MAX_ANN_DT'] == row['MAX_ANN_DT'])]
    same_code_dt_rows = same_code_dt_rows.reset_index(drop = True)
    nan_row = same_code_dt_rows[same_code_dt_rows['分数'].isna()]
    nan_row = nan_row.reset_index(drop = True)
    
    for i in range(len(same_code_dt_rows)):
        if ('本科' in same_code_dt_rows['ITEM_NAME'].loc[i]) or ('大学' in same_code_dt_rows['ITEM_NAME'].loc[i]):
            for j in range(len(nan_rows_new)):
                if nan_row['S_INFO_WINDCODE'].loc[0] == nan_rows_new['S_INFO_WINDCODE'].loc[j] and nan_row['MAX_ANN_DT'].loc[0] == nan_rows_new['MAX_ANN_DT'].loc[j] and nan_row['ITEM_NAME'].loc[0] == nan_rows_new['ITEM_NAME'].loc[j]:
                    nan_rows_new['分数'].loc[j] = 12.5

/var/folders/7b/33w4_t297zxfr68lt1r2hm9c0000gn/T/ipykernel_61324/3764420226.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_rows_new['分数'].loc[j] = 12.5


In [35]:
nan_rows_new

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,其他人员,196,000005.SZ,NaN,2012-04-24
1,其他人员,81,000006.SZ,12.5,2013-03-30
2,其他人员,14,000008.SZ,12.5,2012-03-03
3,其他人员,20,000010.SZ,NaN,2012-04-27
4,其他人员,1206,000011.SZ,NaN,2012-03-30
...,...,...,...,...,...
4165,其他,22,839719.BJ,12.5,2016-07-26
4166,其他,202,870204.BJ,12.5,2016-08-30
4167,其他,65,871970.BJ,12.5,2017-04-28
4168,其他,19,872351.BJ,12.5,2017-05-26


In [20]:
'''for i in range(len(nan_rows_new)):
    for j in range(len(final_df_new)):
        if nan_rows_new['S_INFO_WINDCODE'].loc[i] == final_df_new['S_INFO_WINDCODE'].loc[j] and nan_rows_new['MAX_ANN_DT'].loc[i] == final_df_new['MAX_ANN_DT'].loc[j] and nan_rows_new['ITEM_NAME'].loc[i] == final_df_new['ITEM_NAME'].loc[j]:
            final_df_new['分数'].loc[j] = nan_rows_new['分数'].loc[i]'''

"for i in range(len(nan_rows_new)):\n    for j in range(len(final_df_new)):\n        if nan_rows_new['S_INFO_WINDCODE'].loc[i] == final_df_new['S_INFO_WINDCODE'].loc[j] and nan_rows_new['MAX_ANN_DT'].loc[i] == final_df_new['MAX_ANN_DT'].loc[j] and nan_rows_new['ITEM_NAME'].loc[i] == final_df_new['ITEM_NAME'].loc[j]:\n            final_df_new['分数'].loc[j] = nan_rows_new['分数'].loc[i]"

In [36]:
final_df_new_clean = final_df_new.dropna(subset=['分数'])
final_df_new_clean = final_df_new_clean[['ITEM_NAME', 'STAFF_NUMBER', 'S_INFO_WINDCODE', '分数', 'MAX_ANN_DT']]

In [37]:
final_df_new_clean

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,大专以上,10765,000001.SZ,18.5,2010-03-12
1,本科以上,8040,000001.SZ,19.0,2010-03-12
2,大专,2866,000001.SZ,15.0,2012-03-09
3,其他,454,000001.SZ,12.0,2012-03-09
4,本科及以上,10458,000001.SZ,19.0,2012-03-09
...,...,...,...,...,...
190640,专科,56,873593.BJ,15.0,2022-08-30
190641,本科,75,873593.BJ,16.0,2023-03-18
190642,硕士,5,873593.BJ,18.5,2023-03-18
190643,专科以下,78,873593.BJ,12.0,2023-03-18


In [38]:
merge_df = pd.concat([final_df_new_clean,nan_rows_new], axis = 0).reset_index(drop=True)

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT
0,大专以上,10765,000001.SZ,18.5,2010-03-12
1,本科以上,8040,000001.SZ,19.0,2010-03-12
2,大专,2866,000001.SZ,15.0,2012-03-09
3,其他,454,000001.SZ,12.0,2012-03-09
4,本科及以上,10458,000001.SZ,19.0,2012-03-09
...,...,...,...,...,...
190640,其他,22,839719.BJ,12.5,2016-07-26
190641,其他,202,870204.BJ,12.5,2016-08-30
190642,其他,65,871970.BJ,12.5,2017-04-28
190643,其他,19,872351.BJ,12.5,2017-05-26


In [48]:
# 将NaN值替换为12
merge_df['分数'].fillna(12, inplace=True)

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT,Total


In [49]:
merge_df['Total'] = merge_df['STAFF_NUMBER'] * merge_df['分数']
merge_df

,ITEM_NAME,STAFF_NUMBER,S_INFO_WINDCODE,分数,MAX_ANN_DT,Total
0,大专以上,10765,000001.SZ,18.5,2010-03-12,199152.5
1,本科以上,8040,000001.SZ,19.0,2010-03-12,152760.0
2,大专,2866,000001.SZ,15.0,2012-03-09,42990.0
3,其他,454,000001.SZ,12.0,2012-03-09,5448.0
4,本科及以上,10458,000001.SZ,19.0,2012-03-09,198702.0
...,...,...,...,...,...,...
190640,其他,22,839719.BJ,12.5,2016-07-26,275.0
190641,其他,202,870204.BJ,12.5,2016-08-30,2525.0
190642,其他,65,871970.BJ,12.5,2017-04-28,812.5
190643,其他,19,872351.BJ,12.5,2017-05-26,237.5


In [50]:
#分组并计算总和
grouped = merge_df.groupby(["S_INFO_WINDCODE", "MAX_ANN_DT"]).agg({"Total": "sum", "STAFF_NUMBER": "sum"}).reset_index()
grouped

,S_INFO_WINDCODE,MAX_ANN_DT,Total,STAFF_NUMBER
0,000001.SZ,2010-03-12,351912.5,18805
1,000001.SZ,2012-03-09,247140.0,13778
2,000001.SZ,2012-08-16,382571.0,21417
3,000001.SZ,2013-03-08,434144.0,24251
4,000001.SZ,2013-08-23,461676.0,25754
...,...,...,...,...
47748,873576.BJ,2023-04-01,3682.5,258
47749,873593.BJ,2020-12-30,1258.5,95
47750,873593.BJ,2022-04-28,2379.5,168
47751,873593.BJ,2022-08-30,2802.0,195


In [51]:
grouped['Average'] = grouped['Total'] / grouped['STAFF_NUMBER']
grouped

,S_INFO_WINDCODE,MAX_ANN_DT,Total,STAFF_NUMBER,Average
0,000001.SZ,2010-03-12,351912.5,18805,18.713773
1,000001.SZ,2012-03-09,247140.0,13778,17.937291
2,000001.SZ,2012-08-16,382571.0,21417,17.862959
3,000001.SZ,2013-03-08,434144.0,24251,17.902107
4,000001.SZ,2013-08-23,461676.0,25754,17.926380
...,...,...,...,...,...
47748,873576.BJ,2023-04-01,3682.5,258,14.273256
47749,873593.BJ,2020-12-30,1258.5,95,13.247368
47750,873593.BJ,2022-04-28,2379.5,168,14.163690
47751,873593.BJ,2022-08-30,2802.0,195,14.369231


In [52]:
grouped.to_csv('result.csv', encoding='utf-8_sig', index=True)

In [35]:
# 定义函数来计算与指定日期最接近的日期
def closest_date(date_list, target_date):
    return min(date_list, key=lambda date: date - target_date)

# 指定目标日期
target_dates = [pd.to_datetime('2023-04-30'), pd.to_datetime('2023-10-31')]

# 选出每个公司每年最接近目标日期的数据
selected_data = pd.DataFrame()
for target_date in target_dates:
    for group_name, group_df in grouped_data:
        closest_ann_dt = closest_date(group_df['ANN_DT'], target_date)
        selected_data = pd.concat([selected_data, group_df[group_df['ANN_DT'] == closest_ann_dt]])
        
selected_data

,ITEM_NAME,STAFF_NUMBER,ANN_DT,S_INFO_WINDCODE,分数
262,本科以上,8040,2010-03-12,000001.SZ,19
888,大专以上,10765,2010-03-12,000001.SZ,18.5
19542,本科及以上,10458,2012-03-09,000001.SZ,19
33357,大专,2866,2012-03-09,000001.SZ,15
93626,其他,454,2012-03-09,000001.SZ,NAN
...,...,...,...,...,...
173240,本科,45,2022-04-28,873593.BJ,16
173529,专科,63,2023-03-18,873593.BJ,15.00
173530,专科以下,78,2023-03-18,873593.BJ,12
173531,本科,75,2023-03-18,873593.BJ,16
